# Reverberation datasets

In [1]:
from datasets.Reverberation import Time
from notebooks.notebooks import get_ttv_paths
from transforms.transforms import mfcc

In [2]:
train_path, test_path, validate_path = get_ttv_paths()

train_dataset = Time(train_path, 'train', download=False, transform=mfcc)
test_dataset = Time(test_path, 'test', download=False, transform=mfcc)
validate_dataset = Time(validate_path, 'validate', download=False, transform=mfcc)

In [3]:
print(train_dataset[0])

(array([[-0.45428467, -0.23236084],
       [ 0.1708374 ,  0.619751  ],
       [ 0.07452393, -0.8592529 ],
       ...,
       [ 0.5364075 ,  0.36709595],
       [-0.3960266 ,  0.20367432],
       [ 0.44882202,  0.8918762 ]], dtype=float32), 6)


In [4]:
device = "cpu"
print(f"Using {device}")

Using cpu


In [5]:
def train_network(train, test, validate):
    from torch.utils.data import DataLoader
    from torch import nn
    import torch
    from utils.neural_networks import test_loop, train_loop, validate_loop

    input_size = 1
    for v in train[0][0].shape:
        input_size *= v

    output_size = 9
    batch_size = 64

    learning_rate = 1e-2
    epochs = 5

    train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=True)
    validate_dataloader = DataLoader(validate, batch_size=batch_size, shuffle=True)

    class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(input_size, 512),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(512, output_size)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to(device)
    print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        validate_loop(validate_dataloader, model, loss_fn)
        test_loop(test_dataloader, model, loss_fn)
    print("Done")

## Reverberation time

In [6]:
train_network(train_dataset, test_dataset, validate_dataset)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=18760, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=9, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.223696 [    0/ 6000]
loss: 2.199118 [  640/ 6000]
loss: 2.180701 [ 1280/ 6000]
loss: 2.184377 [ 1920/ 6000]
loss: 2.198511 [ 2560/ 6000]
loss: 2.192051 [ 3200/ 6000]
loss: 2.179449 [ 3840/ 6000]
loss: 2.154261 [ 4480/ 6000]
loss: 2.162158 [ 5120/ 6000]
loss: 2.163050 [ 5760/ 6000]
Validation Error: 
 Accuracy: 12.35, Avg loss: 2.156939 

Test Error: 
 Accuracy: 12.55%, Avg loss: 2.156894 

Epoch 2
-------------------------------
loss: 2.133029 [    0/ 6000]
loss: 2.133557 [  640/ 6000]
loss: 2.132632 [ 1280/ 6000]
loss: 2.123295 [ 1920/ 6000]
loss: